In [22]:
import requests as rq
import csv
import json
from flask import request, jsonify

ModuleNotFoundError: No module named 'flask'

In [56]:
covid_tracking = rq.get("https://covidtracking.com/api/v1/states/current.json").json()

In [17]:
with open("states_coords.csv", "r") as f:
    csv_file = csv.reader(f, delimiter=",")
    csv_unpacked = [row for row in csv_file]
    states_coords = [{csv_unpacked[0][i]: row[i] for i in range(len(row))} for row in csv_unpacked]

In [58]:
for covid_row in covid_tracking:
    for coords_row in states_coords:
        if covid_row['state']==coords_row['state']:
            covid_row['longitude']=coords_row['longitude']
            covid_row['latitude']=coords_row['latitude']

In [21]:
jsonify(covid_tracking)

NameError: name 'jsonify' is not defined

In [4]:
{**covid_tracking[0], 'hi':'yo'}

In [19]:
states_coords[1:]

[{'state': 'AK',
  'latitude': '63.588753',
  'longitude': '-154.493062',
  'name': 'Alaska'},
 {'state': 'AL',
  'latitude': '32.318231',
  'longitude': '-86.902298',
  'name': 'Alabama'},
 {'state': 'AR',
  'latitude': '35.20105',
  'longitude': '-91.831833',
  'name': 'Arkansas'},
 {'state': 'AZ',
  'latitude': '34.048928',
  'longitude': '-111.093731',
  'name': 'Arizona'},
 {'state': 'CA',
  'latitude': '36.778261',
  'longitude': '-119.417932',
  'name': 'California'},
 {'state': 'CO',
  'latitude': '39.550051',
  'longitude': '-105.782067',
  'name': 'Colorado'},
 {'state': 'CT',
  'latitude': '41.603221',
  'longitude': '-73.087749',
  'name': 'Connecticut'},
 {'state': 'DC',
  'latitude': '38.905985',
  'longitude': '-77.033418',
  'name': 'District of Columbia'},
 {'state': 'DE',
  'latitude': '38.910832',
  'longitude': '-75.52767',
  'name': 'Delaware'},
 {'state': 'FL',
  'latitude': '27.664827',
  'longitude': '-81.515754',
  'name': 'Florida'},
 {'state': 'GA',
  'latitu

In [15]:
for state in jf['features']:
    print(state['properties']['NAME'])
    break

Arizona
